# Vector data
This lesson will cover Vector data, giving some theoretical background, introduce
Vector storage formats and finally explore Python libraries for Vector data access and manipulation.

## Background reading

* https://docs.qgis.org/3.16/en/docs/gentle_gis_introduction/vector_data.html

## What is vector data?
Vector data is spatial data, generally consisting of two parts: 

* Geometry
* Attributes

**Geometries** are the *Points, Lines and Polygons* as introduced in the [Geometries Lesson](02-geometry.ipynb).
They represent the "shape" of the real-world phenomenon. 
**Attribute** data is information appended to the Geometry (or the other way around) 
usually in tabular format ("records"). Together, this combination Geometry+Attributes 
is often called a (Spatial) **Feature**.

![Vector Data in QGIS](images/qgis-attr-table.png)

A [Triangulated Irregular network (TIN)](https://en.wikipedia.org/wiki/Triangulated_irregular_network) 
is also an example of Vector data.

## Vector data formats
There are currently [over 100 vector data formats](https://gdal.org/drivers/vector/index.html) used for storage, e.g. files, and for data transfer.
The most common formats are presented below. 

> Tip: [ogr2ogr](https://gdal.org/programs/ogr2ogr.html) is a GDAL/OGR commandline utility
> that allows you to convert between most vector formats.  

### ESRI Shapefile

[ESRI Shapefile](https://en.wikipedia.org/wiki/Shapefile) is a file-based format. It consists of at least 3 files:

* `.shp` containing geometry
* `.shx` containing index
* `.dbf` attribute table

The ESRI Shapefile is one of the oldest formats, some even call it a [Curse in Geoinformatics](https://www.slideshare.net/jachym/switch-from-shapefile), and is more and more replaced by GeoPackage.

### GeoPackage

[GeoPackage](https://www.geopackage.org/) is a relatively new but promising spatial data format based on [SQLite](https://www.sqlite.org).

The [OGC GeoPackage Encoding Standard](https://www.opengeospatial.org/standards/geopackage) describes a set of conventions for storing the following 
within an SQLite database:
  
* vector features
* tile matrix sets of imagery and raster maps at various scales
* attributes (non-spatial data)
* extensions
  
Thus GeoPackage can store vector as well as raster data. GeoPackage is by some called "The Shapefile Killer".
We recommend using GeoPackage over ESRI Shapefile.

### GeoJSON

[GeoJSON](https://geojson.org) is a simple JSON-based format to encode vector Features. 
It is increasingly popular, especially among web developers. 

Example:

```
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
    "population": 4785
  }
}
```

GitHub for example is able to display [GeoJSON-encoded data on-the-fly](https://github.com/jachym/jrdata/blob/master/jsons/stops.geojson). Note that coordinates are always in "easting, northing", thus longitude, latitude here. Note: the use of alternative coordinate reference systems was
removed from an earlier version of the [GeoJSON specification](https://datatracker.ietf.org/doc/html/rfc7946). However: "*...where all involved parties have a prior arrangement, alternative coordinate reference systems can be used without risk of data being misinterpreted.*"

### Geography Markup Language (GML)

> The Geography Markup Language (GML) is the XML grammar defined by the Open Geospatial Consortium (OGC) 
> to express geographical features. GML serves as a modeling language for geographic 
> systems as well as an open interchange format for geographic transactions on the Internet. Source: [Wikipedia](https://en.wikipedia.org/wiki/Geography_Markup_Language).

Below an example of the same feature we saw earlier as GeoJSON, now in GML:

```
<gml:featureMember>
  <feature fid="12">
	<id>23</id>
	<name>Dinagat Islands</name>
	<population>4785</population>
	<ogr:geometry>
	  <gml:Point gml:id="p21" srsName="http://www.opengis.net/def/crs/EPSG/0/4326">
        <gml:pos srsDimension="2">125.6, 10.1</gml:pos>
      </gml:Point>
	</ogr:geometry>
  </feature>
</gml:featureMember>
```

GML is defined as a joint ISO-OGC Standard:

> ISO 19136 Geographic information – Geography Markup Language, is a standard from the family 
> ISO/TC 211 standards for geographic information (ISO 191xx). It resulted from unification 
> of the Open Geospatial Consortium definitions and Geography Markup Language (GML) with 
> the ISO-191xx standards. Source: [Wikipedia](https://en.wikipedia.org/wiki/Geography_Markup_Language)

*GML Application Schemas* adds a convention to the GML standard to define domain- or community- specific application
models. Examples are [CityGML](https://en.wikipedia.org/wiki/CityGML) and schemas developed within [INSPIRE](https://inspire.ec.europa.eu/applicationschema).

GML sees quite widespread use, but due to its complexity (e.g. multiple encodings for coordinates and projections) and verbosity, is more and more
replaced by GeoJSON.

### CSV

Of course, you  can save your data in a comma separated values text file.

### PostgreSQL/PostGIS database

[PostGIS](https://postgis.net) adds support for geographic objects to the PostgreSQL object-relational database. 
In effect, PostGIS "spatially enables" the PostgreSQL server, allowing it to be 
used as a backend spatial database for geographic information systems (GIS), 
much like ESRI's SDE or Oracle's Spatial extension. 
PostGIS follows the OGC [Simple Features Specification for SQL](https://www.opengeospatial.org/standards/sfs) 
and has been certified as compliant with the "Types and Functions" profile. 

Like said, there are [many more vector formats](https://gdal.org/drivers/vector/index.html).

## Vector libraries
Within Python there is an ample choice of libraries to interact with vector data. The
most popular are:

* [Python bindings](https://gdal.org/python/) for [GDAL OGR](https://gdal.org/), a.k.a. "OGR"
* [Fiona](https://toblerity.org/fiona/manual.html) 
* [GeoPandas](https://geopandas.org/) 
 
This chapter will first focus on Fiona and OGR, ending with GeoPandas.
[Fiona](https://toblerity.org/fiona/) is maintained by [Sean Gillies](https://github.com/sgillies) and adds a utility/wrapper layer on top of OGR in a Pythonic fashion.
Compared to Fiona, OGR (Python bindings) provides more finegrained control over data, for example reprojections,
and supports all GDAL/OGR vector formats.

## Manipulating features with Fiona and Shapely
Fiona and Shapely are often used together.
Here we use Fiona 
to read Vector data (Features) into memory for subsequent manipulation with Shapely.

Feature geometry can be accessed using the `geometry` property of each feature. For example
we can open the dataset that contains a (Multi)Polygon for each country and print
out the geometry of a random Feature (country):

First we import `Shapely` and its functions and then convert the JSON-encoded geometries to Geometry objects
using the `shape` function.

In [ ]:
import fiona
from shapely.geometry import shape

Next we open a GeoPackage `countries` file and loop through the Features.
You may observe the Pythonism that Fiona supplies (using `with` and `as`) to
open and loop through Features in a single step.

  > NB the countries-file has its geometries in SRS/CRS EPSG:3857, also known as the
  > [Web Mercator Projection](https://en.wikipedia.org/wiki/Web_Mercator_projection). That projection is
  > in meters, hence values like `area` are in meters as well.

In [ ]:
with fiona.open("../data/countries.3857.gpkg") as countries:
	country = countries[4]
	print(f'This is {country["properties"]["NAME"]}')
	geom = shape(country["geometry"])
  
geom # Jupyter can display geometry data directly

In [ ]:
print(geom.type)

In [ ]:
print(geom.area)

In [ ]:
# In km
print(geom.length / 1000)

Let's have a look at some geometry methods. 
Tip: Shapely code is well-documented, you can always use the Python built-in `help()` function.

In [ ]:
help(geom)

For example we can make a buffer of 500 meter around our polygon (making Canada somewhat bigger):

In [ ]:
buffered_geom = geom.buffer(500)
buffered_geom

In [ ]:
# In km
buffered_geom.length / 1000

We can also create geometry from scratch using various functions in `shapely` as follows

In [ ]:
from shapely.geometry import Polygon,Point,LineString
pt = Point(10,10)
line = LineString([(0,0),(0,3),(3,0)])
poly = Polygon([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]])

In [ ]:
pt

In [ ]:
line

In [ ]:
poly

#### Geospatial analysis using shapely

In [ ]:
poly.touches(line)

In [ ]:
poly.contains(pt)

In [ ]:
pt.buffer(20).contains(poly)

### Converting the geometry back to JSON format
Once we are finished, we can convert the geometry back to JSON format using `shapely.geometry.mapping` function


In [ ]:
from shapely.geometry import mapping

In [ ]:
# let's create new GeoJSON-encoded vector feature

new_feature = {
    'type': 'Feature',
    'properties': {
        'name': 'My buffered feature'
    },
    'geometry': mapping(buffered_geom)
}
new_feature

# Now we could e.g. write the Feature back to file

## GDAL/OGR Python Bindings


[OGR](https://gdal.org/faq.html#what-is-this-ogr-stuff) is part of the [GDAL](https://gdal.org/) library for the support of Vector data. 
OGR supports about [100+ vector formats](https://gdal.org/drivers/vector/index.html) and
has more/other functionalities (than Fiona) like reprojection.

The OGR API wraps differences between various vector formats, web-services, database etc..
The following terminology applies to OGR:

* **Driver** - driver for reading and writing for a specified format
* **Data Source** - the named data source (file, database, web-service, ...)
* **Layer** - data layer within the Data Source (file content, database table, ...)
* **Feature** - vector feature
* **Field, Geometry** - attributes and geometry

The OGR-Python interface is an abstract API on top of the 
original classes and methods of the original C++ code. 
Because of this, some approaches may seem complicated, 
compared to native Python code, like e.g. Fiona.

### Links

* GDAL OGR Vector API tutorial: https://gdal.org/tutorials/vector_api_tut.html
* Python API: https://gdal.org/python/
* GDAL/OGR Python Cookbook https://pcjericks.github.io/py-gdalogr-cookbook/ - Recommended!

### Buffer
First we need to open the *Data Source*, printing the number of Layers.

In [ ]:
from osgeo import ogr
ds = ogr.Open('../data/countries.gpkg')
print(ds)
print(ds.GetLayerCount())

Next we have to fetch and open the *Layer*. NB for files, there is usually just one layer, index `0`, 
within the Data Source (DS), but for example for a database DS, a Layer is refers to a concrete table).

In [ ]:
l = ds.GetLayer(0)
print(l)
print(l.GetFeatureCount())

Show the schema of the layer and the definition of its geometry type:

In [ ]:
l.GetGeomType()

In [ ]:
l.GetGeomType() == ogr.wkbMultiPolygon

In [ ]:
for s in l.schema:
    print(s.GetName())

In [ ]:
l.schema[4].name

Print name attribute of all features

In [ ]:
features_nr = l.GetFeatureCount()
for i in range(features_nr):
    f = l.GetNextFeature()
    print(f.GetField('NAME'))

Get vector feature bounding box (envelope):

In [ ]:
f = l.GetFeature(4)
geom = f.GetGeometryRef()
geom.GetEnvelope()

Get geometry centroid

In [ ]:
c = geom.Centroid()
c.GetPoint()

Get geometry buffer

In [ ]:
buff = c.Buffer(100)
geom.Intersects(buff)

### Complete example

In this example we will demonstrate working with vector data from begin to
end: open a data set, metadata, attribute change, saving of new attribute 
back to the file. 

In [ ]:
from osgeo import osr

# Creating new file with new driver
drv = ogr.GetDriverByName('GML')
ds = drv.CreateDataSource('../data/04-ogr-out.gml')
srs = osr.SpatialReference()
srs.ImportFromEPSG(3857)
print(srs.ExportToProj4())
layer = ds.CreateLayer('outgml', srs, ogr.wkbLineString)

# create new attributes named and code
field_name = ogr.FieldDefn('name', ogr.OFTString)
field_name.SetWidth(24)
field_number = ogr.FieldDefn('code', ogr.OFTInteger)
layer.CreateField(field_name)
layer.CreateField(field_number)

# create new line geometry and read from WKT
line = ogr.CreateGeometryFromWkt('LINESTRING(%f %f, %f %f)' % (0, 0, 1, 1))

# create new feature, set attributes and geometry
feature = ogr.Feature(layer.GetLayerDefn())
feature.SetGeometry(line)
feature.SetField('name', 'the line')
feature.SetField('code', 42)

layer.CreateFeature(feature)

# final cleaning
feature.Destroy()
ds.Destroy()

now we can check the result

In [ ]:
ds = ogr.Open('../data/04-ogr-out.gml')
layer = ds.GetLayer(0)
print(layer.GetFeatureCount())
print(layer.GetFeature(0).GetField('name'))
print(layer.GetFeature(0).GetField('code'))
ds.Destroy()

## Fiona or GDAL/OGR?
With Fiona, the above example would be much simpler and Pythonic. 
However, OGR accesses the data on a much lower/efficient level compared to Fiona, 
therefore bigger datasets can be more easily handled. Also OGR supports more data formats and
functionality like reprojection.

We recommend to have both Fiona (plus Shapely) and OGR in your toolbox
and assess at project-time which to apply.

## Introduction to GeoPandas

Geospatial data is often available from specific GIS file formats or data stores, like ESRI Shapefiles, GeoJSON files,
GeoPackage files, PostGIS (PostgreSQL) databases, ...

We can use the [GeoPandas](https://geopandas.org/) library to read many of those GIS
formats (relying on the `Fiona` library under the hood, which is an interface
to GDAL/OGR), using the `geopandas.read_file` function.

### What's a GeoDataFrame?

We used the GeoPandas library to read in the geospatial data. This returns a `GeoDataFrame`:

A GeoDataFrame contains a tabular, geospatial dataset, basically a Feature collection/record-set:

* It has a **'geometry' column** that holds the geometry information.
* The other columns are the **attributes** that describe each of the geometries

Such a `GeoDataFrame` is just like a pandas `DataFrame`, but with some additional functionality for working with geospatial data:

* A `.geometry` attribute that always returns the column with the geometry information (returning a GeoSeries). The column name itself does not necessarily need to be 'geometry', but it will always be accessible as the `.geometry` attribute.
* It has some extra methods for working with spatial data (area, distance, buffer, intersection, ...), which we will see in later notebooks

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas as gpd

pd.options.display.max_rows = 10

### Loading Data

First step is to load the data into Python.
This data can be a local file, data stored in database, or a file hosted on some server.
Basically any of the (Vector) data formats we introduced above.

#### Loading a Shapefile 

Loading all countries geometry (src: https://www.naturalearthdata.com/downloads/10m-cultural-vectors/)

In [ ]:
#load it as a pandas dataframe with with geometry data
countries = gpd.read_file('../data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')

countries

In [ ]:
countries.head()

In [ ]:
countries.tail(10)

In [ ]:
countries.plot()

In [ ]:
places = gpd.read_file('../data/populated_places.gpkg')

places

#### Loading a GeoJSON file

Loading local geojson file 

In [ ]:
rivers = gpd.read_file('../data/rivers.geojson')
rivers

#### Loading PostgreSQL table

Loading data from a database. (we only show the code, you may want to run
and populate a local PostGIS-enabled database).

```python
import psycopg2 

con = psycopg2.connect(database="postgres", user="postgres", password="postgres",
    host="localhost")

sql = "SELECT * FROM public.places"
places = gpd.read_postgis(sql, con, geom_col='geom')
```

#### Importing a CSV file

Assuming that a CSV has a geometry column in OGC Well-Known Text (WKT) format:

In [ ]:
from shapely import wkt

airport = gpd.read_file('../data/airport.csv')

airport['geometry'] = airport['geom'].apply(wkt.loads)
del airport['geom']
airport

#### Creating a geometry on the fly

Create a Geodataframe from a CSV having standard text columns like 
'longitude' or 'lon' and 'latitude' or 'lat'. 
These which will be used to create a geometry column in the GeoDataFrame on-the-fly.

In [ ]:
import pandas as pd
df = pd.read_csv('../data/stadium.csv')
stadium = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat))
stadium

#### Creating a GeoDataFrame manually 

You can also create a GeoDataFrame in the Jupyter notebook, using your own data:

In [ ]:
from shapely.geometry import Point

parking_enforcement = gpd.GeoDataFrame({
    'id': [1, 2,3,4,5],
    'geometry': [Point(1, 1), Point(2, 2),Point(2, 1),Point(1, 2),Point(1.5, 2)],
    'parking_meters': [12,34,112,41, 212]})
parking_enforcement

### Handling CRS in Geopandas

Unlike `Shapely`, `GeoPandas` understands CRS concepts.

Why are CRSs important?
 
- a CRS definition will provide a standard mechanism to communicate the projection information of a given dataset.  This ensures correct and accurate placement of geometry
- CRS will make sense out of your data such as whether the units are degrees/meters
- Bringing all data in the same CRS allows us to do spatial analysis with data 


Display the CRS of a GeoDataFrame

In [ ]:
countries.crs

We can also set CRS for the GeoDataFrame which has no default CRS

In [ ]:
parking_enforcement = parking_enforcement.set_crs('epsg:4326')
parking_enforcement.crs

We can also convert a GeoDataFrame from one CRS to another

In [ ]:
parking_enforcement.crs

In [ ]:
parking_enforcement_3857 = parking_enforcement.to_crs(3857)

In [ ]:
parking_enforcement_3857.crs

### Merging DataFrames

#### Attribute based merge

In [ ]:
neighbor = pd.DataFrame({
    'id': [1, 2,3,4,5],
    'neighbor_id': ['a1', 'a2','b3','c4','d5'],
    'neighbor_name': ['andy','julio','droid','steve', 'ramesh']})
neighbor

In [ ]:
updated_parking_enforcement = parking_enforcement.merge(neighbor, on='id')
updated_parking_enforcement

#### Spatial merge

GeoPandas also provides functions for "Spatial Joins".
We won't go in many details here, but it is a very powerful
feature you may want to check out. Some examples below:


```python
pd.set_option('display.max_columns', 100)
airport = airport.set_crs('epsg:4326')
airport.head()

simple_countries = countries[['ADMIN','geometry']]
simple_countries.head()

airport_with_country = gpd.sjoin(airport, simple_countries, how="inner", op='intersects')
airport_with_country.head()

```

The `op` parameter is another way to perform same query can be using
 operation `within` instead of `intersect`:

```python
airport_with_country_within = gpd.sjoin(airport, simple_countries, how="inner", op='within')
airport_with_country.head()

```
Further parameters:

The `how` parameter. We can use `left` , `right` , `inner` .

* `left`: use the index from the first (or left_df) geodataframe that you provide to sjoin; retain only the left_df geometry column
* `right`: use the index from second (or right_df); retain only the right_df geometry column
* `inner`: use the intersection of index values from both geodataframes; retain only the left_df geometry column

```python
airport_with_country_right = gpd.sjoin(airport, simple_countries, how="right", op='within')
airport_with_country_right.head()
```

### Edit the existing data

#### Editing metadata

In [ ]:
updated_parking_enforcement.iloc[0]

In [ ]:
updated_parking_enforcement.iloc[0,2] = 24

In [ ]:
updated_parking_enforcement.iloc[0]

#### Editing geometry

In [ ]:
from shapely.geometry import Point

updated_point = Point(3,4)
updated_parking_enforcement.iloc[0,0] = updated_point
updated_parking_enforcement

### Querying data

#### Based on metadata

In [ ]:
countries.head()

In [ ]:
India = countries[countries['ADMIN'] == "India"]
India

In [ ]:
densly_pop = countries[countries['POP_EST'] > 100000000]
densly_pop

In [ ]:
countriesWithC = countries[countries['SOVEREIGNT'].str.startswith('C')]
countriesWithC

In [ ]:
densecountriesWithC = countries[(countries['SOVEREIGNT'].str.startswith('C')) &  (countries['POP_EST'] > 1000000000)]
densecountriesWithC

#### Spatial query

Spatial query uses shapely geometry as base geometry on top of which geodataframe can be queried.
Available oprations are listed at
https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates

In [ ]:
indian_shape = India['geometry'].squeeze()

In [ ]:
type(India['geometry'].squeeze())

In [ ]:
test_pt = Point(1,1)

In [ ]:
test_pt.intersects(indian_shape)

In [ ]:
nashik = Point(73.76,19.93)

In [ ]:
nashik.within(indian_shape)

In [ ]:
indian_airport = airport[airport.within(indian_shape)]
indian_airport

#### Quiz: Can you create the dataframe of all airports and cities within your country

In [ ]:
indian_rivers = rivers[rivers.intersects(indian_shape)]
indian_rivers.plot()

In [ ]:
Neighbours_India = countries[countries.touches(indian_shape)]
Neighbours_India.plot()

### Geospatial Operations

Understanding base logic first! Back to Shapely.

In [ ]:
test_point = Point(0,0)
test_point

In [ ]:
test_point.buffer(10)

In [ ]:
test_point.buffer(10).area

In [ ]:
from shapely.geometry import LineString

test_line = LineString([(0, 0), (1, 1), (0, 2)])
test_line

In [ ]:
#Buffer puts original geometry at center and create buffer alongside
test_line.buffer(0.1)

In [ ]:
#We can also put geometry on either side ( Positive value will put buffer to left)

test_line.buffer(0.5, single_sided=True)

In [ ]:
#We can also put geometry on either side ( negative value will put buffer to right)

test_line.buffer(-0.5, single_sided=True)

#### Operations in `geopandas`

In [ ]:
Indian_cities =  places[places.within(indian_shape)]
Indian_cities

In [ ]:
Indian_cities_m = Indian_cities.to_crs(3857)
Indian_cities_m.crs

In [ ]:
city_buffer = Indian_cities_m[['geometry','NAME']]
city_buffer

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(16, 16))
India.plot(ax=ax, color='#ffffff', edgecolor='#6a6a6a', linewidth=2)
city_buffer.plot(ax=ax, color='#f00', edgecolor='#000000')

In [ ]:
city_buffer['geom'] = city_buffer.buffer(50000)
city_buffer

In [ ]:
countries_centroid = countries[['geometry','NAME','CONTINENT']]
countries_centroid.head()

In [ ]:
countries_centroid['geometry'] = countries_centroid['geometry'].centroid
countries_centroid.head()

In [ ]:
countries['area'] = countries['geometry'].area
countries.head()

## Visualising a GeoDataFrame

In [ ]:
#simple visualisation 
countries.plot()

In [ ]:
countries_m = countries[countries['NAME'] != "Antarctica"]
countries_m.plot()

In [ ]:
#color based on column
countries_m.plot(column='CONTINENT')

In [ ]:
countries_m.plot(column='CONTINENT',legend=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 16))
countries_m.plot(ax=ax,column='CONTINENT',legend=True)

In [ ]:
ax = countries_m.plot(column='CONTINENT',legend=True)
ax.set_axis_off()

In [ ]:
#Checkout available color maps => https://matplotlib.org/2.0.2/users/colormaps.html
countries_m.plot(column='CONTINENT',  cmap='winter')


In [ ]:
countries_plot = countries_m[(countries_m['NAME'] != 'India') & (countries_m['NAME'] != 'China')]
countries_plot.plot(column='POP_EST',legend=True,figsize=(16,16), legend_kwds={'label': 'Population'})

### matplotlib to show multiple data 

In [ ]:
basemap = countries_m.plot(column='CONTINENT', cmap='cool')
cities_m = places.to_crs(3857)
cities_m.plot(ax=basemap, marker='o', color='red', markersize=5)

In [ ]:
#load world polygon
bbox = gpd.read_file('../data/world.geojson')
world = bbox.loc[0].geometry
world

In [ ]:
cities_m = cities_m[cities_m.within(world)]

In [ ]:
basemap = countries_m.plot(column='CONTINENT', cmap='cool')
cities_m.plot(ax=basemap, marker='o', color='red', markersize=5)

### geopandas overlay to show multiple data 

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))

India.plot(ax=ax, color='b', edgecolor='#f0f', linewidth=2)
Indian_cities_m.plot(ax=ax, color='r', edgecolor='#fff')


In [ ]:
Indian_cities_m['geometry'] = Indian_cities_m['geometry'].buffer(50000)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))

India.plot(ax=ax, color='b', edgecolor='#f0f', linewidth=2)
Indian_cities_m.plot(ax=ax, color='r', edgecolor='#fff')

You may also do an `overlay` function:

```
non_rural_area = gpd.overlay(India, Indian_cities_m, how='difference')
non_rural_area.plot(figsize=(16, 16))
```

---
[<- Spatial Reference Systems](03-spatial-reference-systems.ipynb) | [Raster data ->](05-raster-data.ipynb)
